# TME 2 : Prédiction de gènes



## Partie 1 : Détection de gènes candidats


Le but de cette partie est d’annoter les régions dans le génome qui correspondent à des gènes. Nous allons prendre en compte les différents éléments qui définissent un gène pour pouvoir déterminer les candidats à des régions codantes : phases ouvertes de lecture, propriétés statistiques du code génétique, et comparer les résultats avec l’annotation qui est disponible. 
Nous nous baserons sur le génome de _Bifidobacterium actinocoloniiforme_ que nous avons commencé à analyser au TME précédent, avec ses annotations et au génome de _Escherichia coli_


+ (mise en route) Adapter les commandes du TME précédent pour télécharger le génome de _E. coli_ ainsi que ses annotations

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_genomic.fna.gz

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/026/345/GCF_000026345.1_ASM2634v1/GCF_000026345.1_ASM2634v1_feature_table.txt.gz


In [2]:

##Dictionnaire codons -> acides aminés. Les codons stops sont représentés
##avec le caractère "*"
codeGenetique = {"TTT":"F","TTC":"F","TTA":"L","TTG":"L","TCT":"S","TCC":"S",
				"TCA":"S","TCG":"S","TAT":"Y","TAC":"Y","TAA":"*","TAG":"*",
				"TGT":"C","TGC":"C","TGA":"*","TGG":"W","CTT":"L","CTC":"L",
				"CTA":"L","CTG":"L","CCT":"P","CCC":"P","CCA":"P","CCG":"P",
				"CAT":"H","CAC":"H","CAA":"Q","CAG":"Q","CGT":"R","CGC":"R",
				"CGA":"R","CGG":"R","ATT":"I","ATC":"I","ATA":"I","ATG":"M",
				"ACT":"T","ACC":"T","ACA":"T","ACG":"T","AAT":"N","AAC":"N",
				"AAA":"K","AAG":"K","AGT":"S","AGC":"S","AGA":"R","AGG":"R",
				"GTT":"V","GTC":"V","GTA":"V","GTG":"V","GCT":"A","GCC":"A",
				"GCA":"A","GCG":"A","GAT":"D","GAC":"D","GAA":"E","GAG":"E",
				"GGT":"G","GGC":"G","GGA":"G","GGG":"G"}

#Utliser la sequence testSeq pour tester vos fonctions
testSeq = "ATGAAACGCATTAGCMMCACCATTACCACCACCATCACCATTACCACAGKTAACGGTGCGGGCTGA"

### A) Annotation des phases ouvertes de lecture et traduction en séquence protéique

<b>Question 1)</b> Parfois dans les génomes (représentés par des fichiers numériques), nous trouvons des nucléotides qui n'ont pas été correctement identifiés par la machine de séquençage. Certains instruments nous restreignent les possibilités à un sous-ensemble de nucléotides. Ecrire une fonction `remplaceNonIden` qui remplace les nucléotides non identifiés par une de possibilités listées ci-desous de façon aléatoire.<br>

R = G,A (purine)<br>
Y = T,C (pyrimidine)<br>
K = G,T (céto)<br>
M = A,C (amino)<br>
S = G,C (obligations solides)<br>
W = A,T (Les liaisons faibles)<br>
B = G,T,C (tous sauf A)<br>
D = G,A,T (tous sauf C)<br>
H = A,C,T (tous sauf G)<br>
V = G,C,A (tous sauf T)<br>
N = A,G,C,T (any)<br>
X = A,G,C,T (any)<br>



In [5]:
# Question 1 
def remplaceNonIden(seq):
    seq_cleaned = ""
    return seq_cleaned

testSeqClean =  remplaceNonIden(testSeq)
print (testSeqClean)
#Resultat attendu
#"ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA"

<b>Question 2)</b> Ecrire une fonction `listeCodon` qui renvoie une liste de codons pour une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``[‘AAC’, ‘GTG’, ‘GCA’]``. Si la longueur de la séquence n'est pas un multiple de 3 on ne tiendra pas compte des 1 ou 2 nucléotides restant à la fin.


In [6]:
# Question 2 
def listeCodon(seq):
    lcodons = []
    return lcodons

#Resultat attendu
lcodons = listeCodon(testSeqClean)
print (lcodons)
#Resultat attendu
#['ATG', 'AAA', 'CGC', 'ATT', 'AGC', 'ACC', 'ACC', 'ATT', 'ACC', 'ACC', 'ACC', 'ATC', 'ACC', 'ATT', 'ACC', 'ACA', 'GGT', 'AAC', 'GGT', 'GCG', 'GGC', 'TGA']

[]


<b>Question 2)</b> Ecrire une fonction `listeORFS` qui retourne la liste de tous les cadres ouverts de lectures, c'est à dire les séquence commençant par un des codons start: ``ATG``, ``GTG``, ``TTG`` (``ATG`` est le plus fréquent) et finissant par un des codons stop : ``TAA``, ``TAG``, ``TGA``. Vous renverrez les séquences pour les 6 phases de lecture (3 pour le brin sens et 3 pour le brin complémentaire). Ces séquences sont nommées séquences CDS (pour _CoDing Sequences_ en anglais) et les phases ouvertes de lecture ORFS (pour _Open Reading Frame_). <br>
Note 1 : si plusieurs start sont trouvés dans la même phase de lecture qu'un stop et en amont de celui ci, le CDS débute par conventions au start le plus en amont. <br>
Note 2 : les CDS peuvent se chevaucher (sur différentes phases et éventuellement sur le brin d'ADN complémentaire).


In [7]:

# Question 3
def listeORFS(seq):
    orfs = []
    return orfs

orfs = listeORFS(testSeqClean)
print (orfs)
#Resultat attendu
#"TCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCAT"


[]


<b>Question 4)</b> Ecrire une fonction `reversecompl` qui renvoie le brin complémentaire d’une séquence passée en paramètre. Par exemple, si on passe la séquence ``AACGTGGCA`` comme paramètre votre fonction doit renvoyer ``TGCCACGTT``.


In [8]:
# Question 4
def reversecompl(seq):
    revseq = ""
    return revseq
revseq = reversecompl(testSeqClean)
print (revseq)
#Resultat attendu
#"TCAGCCCGCACCGTTACCTGTGGTAATGGTGATGGTGGTGGTAATGGTGGTGCTAATGCGTTTCAT" 

<b>Question 5)</b> Développez une fonction `traduitProt` qui étant donné un gène renvoie la protéine traduite. Utilisez le dictionnaire définit ci dessous.

In [9]:
# Question 5
def traduitProt(seq, codeGenetique):
    tradseq = ""
    return tradseq

tradseq = traduitProt(testSeqClean, codeGenetique)
print (tradseq)
#Resultat attendu
#"KRISTTITTTITITTGNGAG*"

### B) Analyse des phases ouvertes de lecture

<b>Question 1)</b> Utilisez votre fonction `listeORFS` qui trouve toutes les ORFs pour ne trouver que les ORFs d’un génome sur le brin sens, c'est à dire pour trois premières phases. Sauvegarder l'ensemble des ORFs trouvé dans un fichier de type fasta.


In [ ]:
# Question 1
# charger le fichier fasta du génome E. coli
# extraire les ORFs
# sauvegarder un fichier fasta avec les ORFs


<b>Question 2)</b> Dessinez un histogramme (avec matplotlib) de la distribution de longueur des ORFs détectés précédemment, que remarquez vous ? 


In [ ]:
# Question 2
# Plot avec matplotlib
import matplotlib.pyplot as pl

lORFS = listORFs(genome)
# calculer les longueurs des ORFs
lenORFs = map(len, lORFS)

# Affichage de l'histogramme
nbins= 50
pl.figure()
pl.hist(lenORFs, nbins)


<b>Question 3)</b> Utilisez la question précédente pour fixer un seuil pour dire qu’un ORF est un gène. (par exemple plus que 99 nucléotides). En déduire une première annotation des gènes dans le génome de _Escherichia coli_. Vous produirez un fichier tab avec les positions de tous les ORFs annotés. Quels gènes du brin positif vont être ratés à votre avis ? 

In [ ]:
# Question 3
# Fichier d'annotation des ORFs


### Partie 2 : Evaluation des résultats

#### A) Matrice de confusion et mesures de performance 

Il est nécessaire d’avoir une méthode pour évaluer la qualité des prédictions obtenues pour les coordonnées des gènes. Est ce que certains gènes sont ratés par la prédiction, ou est-ce qu’au contraire certains des gènes sont systématiquement surannotés alors qu’il s’agit d’une région intergénique (entre les gènes) ? Toute l’information peut être résumée en construisant le tableau de valeurs suivante, appelée matrice de confusion. Pour cela nous attribuons à chaque nucléotide d’un génome donné la valeur 0 ou 1, 0 si le nucléotide ne fait pas partie d’un gène et 1 si il en fait partie. La matrice de confusion est calculée comme ce tableau. 

![](matriceconfusion.png "")

On résume ensuite la qualité de l’annotation avec les métriques suivantes : 

   + La sensibilité  (= ``11/(11+10)``) <br>
Sen = (le taux de vrais positifs) / (le taux de vrais positifs + le taux de faux négatifs)

   + La spécificité (= ``00 / (00+01)``) <br>
Spe = (le taux de vrais négatifs) / (le taux de vrais négatifs + le taux de faux positifs)

   + La valeur prédictive VP (= ``11 / (11+01)``) est le taux de vrais positifs parmi ceux prédits comme positifs <br>
VP = (le taux de vrais positifs) / (le taux de vrais positifs + le taux de faux positifs)


<b>Question 4)</b> Ecrivez une fonction qui calcule la Sen et la Spe et VP à partir d’une matrice de confusion et renvoie un tuple à trois dimensions. La matrice de confusion sera codée comme un tableau à deux dimension, _e.g._ une liste de listes.



In [ ]:
# Question 4
def evalRes(matconf):
    return (sen,spe,vp)


<b>Question 5)</b> Ecrivez une fonction ``ecrit_intervalle`` qui, à partir des deux listes des positions de début et de fin des gènes sur le brin positif et de la longueur du génome, recode une grande liste avec la valeur 1 pour les positions qui sont des gènes et 0 sinon. Attention, il faut tenir compte des cas possibles où des gènes se chevauchent. Vous pourrez par exemple initialiser une liste avec ``lg`` valeurs à ``0`` et traiter ensuite les intervalles séquentiellement.


In [11]:

# Question 5
def ecrit_intervalle(posdeb, posfin, lgenome):
    lcode =[]
    return lcode

# exemple 
lg = 22
positions_debut = [3,11,16]
positions_fin = [8,13,21] 

li = ecrit_intervalle(positions_debut, positions_fin, lg)
# On doit avoir li égal à [0,0,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0]. 




<b>Question 6)</b> Ecrivez la fonction ``compare_intervalle`` qui compare deux listes produites par la fonction ``ecrit_intervalle``, et renvoie la matrice de confusion. <br>
    Par exemple, si les deux listes données en paramètre sont <br> 
    ``génome =[0,0,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0]`` et <br>
    ``ORFs   =[0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0]``, on a :<br>

|            | ORFs (*0*) | ORFs (*1*) |
|------------|----------|----------|
| Genome (*0*) | 7        | 0        |
| Genome (*1*) | 4        | 11       |

In [ ]:
# question 6
def compare_intervalles(int1, int2):
    return 

### B) Evaluation sur _E. coli_

<b>Question 1)</b> Appliquez ``compare_intervalle`` à l’annotation des gènes sur le brin positif et à la prédiction par les ORFs pour générer la matrice de confusion. Et calculez ensuite la Sen et la Spe et VP pour évaluer l’annotation. **Attention, certains gènes définit dans les fichiers .tab sont non-codant (rRNA transfer, rRNA ribossomal). Ces gènes ne commencent pas par un codon START et  nous allons les ignorer dans notre comparaison**. Donnez les résultats pour l’annotation obtenue à la question précédente pour le génome de Coli. Est ce que cela vous semble être un bon résultat ?

<b>Question 2)</b> Analyse des résultats. Pour comprendre les limitations d'une méthode de prédiction on doit toujours analyser dans un second temps les erreurs faites par le programme. En détectant des erreurs récurrente, il est plus facile de proposer ensuite des améliorations. Donnez deux exemples de régions d'au moins 50 nucléotides consécutifs qui sont :
    - un faux négatif (gène raté)
    - un faux positif (gène inexistant)
A votre avis à quoi ces deux types d'erreurs sont-elles dues ? 


In [ ]:
# Question 1

In [ ]:
# Question 2